<a href="https://colab.research.google.com/github/kks0507/multiple_text_documents_chatbot/blob/main/multi_document_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 여러 텍스트 문서와 대화할 수 있는 챗봇

In [ ]:
!pip install -q langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 여러 문서 가지고 오기

In [ ]:
# wget 명령어로 GitHub에서 파일을 다운로드 (techcrunch-articles.zip 파일)
!wget -q https://github.com/kairess/toy-datasets/raw/master/techcrunch-articles.zip

# 다운로드된 techcrunch-articles.zip 파일을 articles 폴더에 압축 해제
!unzip -q techcrunch-articles.zip -d articles

### 1단계. 랭체인 설정하기

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
!pip install -U langchain-community # install the langchain_community package

from langchain.vectorstores import Chroma # This should work after installing langchain-community
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


### 2단계. 다수의 텍스트 파일 로드 후 저장하기

In [ ]:
# DirectoryLoader 클래스를 사용하여 articles 폴더에서 텍스트 파일들을 로드
# glob="*.txt"는 폴더 내에서 확장자가 .txt인 파일들만 로드함을 의미
# loader_cls=TextLoader는 각 텍스트 파일을 개별적으로 처리하는 로더 클래스로 TextLoader를 사용
loader = DirectoryLoader('./articles', glob="*.txt", loader_cls=TextLoader)

# 하나의 텍스트 파일만 로드하는 경우, loader = TextLoader('single_text_file.txt')

# 텍스트 파일을 모두 로드하여 documents 리스트에 저장
documents = loader.load()

# 로드된 문서의 개수를 출력
len(documents)

21

### 3단계. 텍스트들을 1000글자 씩 분할하기

#### documents 리스트에 있는 문서를 일정 크기의 조각으로 나누어, 총 몇 개의 조각이 생성되었는지 확인하는 작업을 수행합니다.

In [ ]:
# RecursiveCharacterTextSplitter 클래스를 사용해 문서를 쪼갤 준비
# chunk_size=1000은 한 덩어리의 최대 문자 수를 1000자로 설정
# chunk_overlap=200은 덩어리들이 겹치는 부분을 200자로 설정, 이전 덩어리와 일부 중복되게끔 만듦
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# documents 리스트에 있는 각 문서를 설정된 chunk_size와 chunk_overlap을 기준으로 분할
texts = text_splitter.split_documents(documents)

# 분할된 텍스트 덩어리(chunk)의 개수를 출력
len(texts)

233

In [ ]:
# texts 리스트에서 인덱스 2부터 4 이전까지의 요소(즉, 2번과 3번 요소)를 출력
texts[2:4]

[Document(metadata={'source': 'articles/05-05-with-deepfloyd-generative-ai-art-gets-a-text-upgrade.txt'}, page_content='With a typical diffusion model, the model learns how to gradually subtract noise from a starting image made almost entirely of noise, moving it closer step by step to the target prompt. DeepFloyd IF performs diffusion not once but several times, generating a 64x64px image then upscaling the image to 256x256px and finally to 1024x1024px.\n\nWhy the need for multiple diffusion steps? DeepFloyd IF works directly with pixels, Russell explained. Diffusion models are for the most part latent diffusion models, which essentially means they work in a lower-dimensional space that represents a lot more pixels but in a less accurate way.'),
 Document(metadata={'source': 'articles/05-05-with-deepfloyd-generative-ai-art-gets-a-text-upgrade.txt'}, page_content='The other key difference between DeepFloyd IF and models such as Stable Diffusion and DALL-E 2 is that the former uses a la

### 4단계. 텍스트 chunk들을 임베딩을 해준 이후 Chroma DB에 저장

In [ ]:
# 벡터 데이터베이스를 저장할 디렉터리 경로 설정
persist_directory = 'db'

# OpenAIEmbeddings 객체 생성, 텍스트 데이터를 벡터로 변환하는 데 사용
embedding = OpenAIEmbeddings()

# Chroma 벡터 데이터베이스를 생성하여 텍스트 문서를 임베딩으로 변환
vectordb = Chroma.from_documents(
    documents=texts,  # 분할된 텍스트 조각 리스트를 DB로 변환할 document
    embedding=embedding,  # 텍스트 임베딩을 생성하는 데 사용할 OpenAIEmbeddings 객체
    persist_directory=persist_directory  # 벡터 데이터베이스를 저장할 디렉터리
)

#### 4.1) 벡터 DB 초기화 방법

In [ ]:
vectordb.persist()
vectordb = None

#### 4.2) 벡터 데이터베이스 로드하는 예시

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

### 5단계. Retriever를 사용해서 벡터 데이터베이스에서 연관된 문서 찾기


In [ ]:
retriever = vectordb.as_retriever()

##### 이 코드는 **생성형 AI(Generative AI)**에 대한 질문을 바탕으로 검색된 문서들의 출처를 출력하는 기능을 합니다.


#####retriever.get_relevant_documents("What is Generative AI?"): 검색 기능을 제공하는 retriever 객체를 사용해 **"What is Generative AI?"**라는 질문에 대해 관련 문서를 검색합니다. 이 함수는 관련 문서들의 리스트를 반환합니다.

#####for doc in docs: 검색된 문서 리스트(docs)를 반복하며 각 문서에 접근합니다.

#####doc.metadata["source"]: 각 문서의 메타데이터에서 "source" 필드를 가져와, 해당 문서의 출처 또는 경로 정보를 출력합니다.

In [ ]:
# 쿼리 "What is Generative AI?"에 대한 관련 문서를 검색 (존재하는 모든 결과 반환)
docs = retriever.get_relevant_documents("What is Generative AI?")

# 검색된 문서들에 대해 반복
for doc in docs:
    # 각 문서의 메타데이터에서 "source" 필드 값을 출력 (문서의 출처를 나타냄)
    print(doc.metadata["source"])

<ipython-input-12-e216ad9a50a9>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is Generative AI?")


articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt
articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt
articles/05-03-spawning-lays-out-its-plans-for-letting-creators-opt-out-of-generative-ai-training.txt


#### 5-1) 결과를 k개 반환

In [ ]:
# Chroma 벡터 데이터베이스(vectordb)에서 검색 기능을 수행하는 retriever 객체 생성
# search_kwargs={"k": 3}는 검색 결과로 가장 관련성 높은 3개의 문서를 반환하도록 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# "What is Generative AI?"라는 질문에 대해 관련 문서를 검색
docs = retriever.get_relevant_documents("What is Generative AI?")

# 검색된 문서들에 대해 반복
for doc in docs:
    # 각 문서의 메타데이터에서 "source" 필드 값을 출력 (문서의 출처를 나타냄)
    print(doc.metadata["source"])

articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt
articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt


### 6단계. ChatGPT 기반의 chain 형성

In [ ]:
# RetrievalQA 객체를 생성하여 검색 기반 질의응답 시스템을 만듦
# RetrievalQA.from_chain_type: 검색 기반 질의응답(QA) 시스템을 생성하는 함수입니다. 사용자가 쿼리를 입력하면 관련 문서를 검색하고, 그 문서를 기반으로 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # OpenAI의 언어 모델을 사용하여 답변 생성
    chain_type="stuff",  # 사용될 체인 유형, 여기서는 'stuff' 방식을 사용 (모든 문서를 한 번에 요약)
    retriever=retriever,  # 검색을 수행하는 retriever 객체를 연결 (관련 문서를 검색하는 기능 담당)
    return_source_documents=True  # 결과와 함께 원본 문서도 반환하도록 설정
)

In [ ]:
# LLM의 응답을 처리하고 결과와 출처를 출력하는 함수
def process_llm_response(llm_response):
    # 응답의 'result' 키에서 생성된 답변을 출력
    print(llm_response['result'])

    # 출처 문서를 출력하기 위한 구분선
    print('\n\nSources:')

    # 'source_documents' 키에 저장된 문서들의 출처를 반복하며 출력
    for source in llm_response["source_documents"]:
        # 각 문서의 메타데이터에서 'source' 필드(출처)를 출력
        print(source.metadata['source'])

### 7단계. Query

In [ ]:
query = "How much money did Pando raise?"
# llm_response = qa_chain(query): 생성된 쿼리를 qa_chain에 전달하여 질의응답 시스템을 통해 답변을 생성합니다.
llm_response = qa_chain(query)
# process_llm_response(llm_response): process_llm_response 함수에 llm_response를 전달하여 LLM이 생성한 답변과 그 출처 문서를 출력합니다.
process_llm_response(llm_response)

<ipython-input-21-02cd645ef8ec>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = qa_chain(query)


 Pando raised $30 million in its Series B round, bringing its total raised to $45 million.


Sources:
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
llm_response

{'query': 'How much money did Pando raise?',
 'result': ' Pando raised $30 million in its Series B round, bringing its total raised to $45 million.',
 'source_documents': [Document(metadata={'source': 'articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}, page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business 

In [ ]:
query = "Who led the round in Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Iron Pillar and Uncorrelated Ventures led the round.


Sources:
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
query = "Who is CMA?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


CMA stands for Competition and Markets Authority, which is a government body in the UK responsible for promoting competition and preventing anti-competitive activities in markets.


Sources:
articles/05-04-cma-generative-ai-review.txt
articles/05-04-cma-generative-ai-review.txt
articles/05-04-cma-generative-ai-review.txt
